In [225]:
#Dependencies 
import pandas as pd 

#Load the file
file = 'Resources/purchase_data.csv'
purchase_data = pd.read_csv(file)

In [226]:
#Display the first five rows to understand data 
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [227]:
#Create a value count for total players in the table
#Since I will be using this, keep it concise and drop all null values first.

purchase_data_df = purchase_data.dropna(how="any")
purchase_data_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [228]:
#Of the total players, create a dataframe
#The dataframe identified is singular, simply a count of all players by the SN column
#Summarize the values in the column and print

purchase_data_df_totalplayers = purchase_data['SN']
len(purchase_data_df_totalplayers.value_counts())
total_number_of_players = len(purchase_data_df_totalplayers.unique())

# purchase_data_df_totalplayers

pd.DataFrame({
    'Total Players' : [total_number_of_players]
})

,Total Players
0,576


In [229]:
#define unique player items 

player_unique_items = purchase_data['Item ID']

#calc total of unique player items 

number_of_unique_items_in_df = len(player_unique_items.unique())
number_of_unique_items_in_df

183

In [230]:
#define purchase price 

purchase_price = purchase_data['Price']

#define count of all rows 
#calc average price within price column

average_price = purchase_price.mean()
AveragePriceinUSD = ('${:,.2f}'.format(average_price))
AveragePriceinUSD

'$3.05'

In [231]:
#define number of purchases

number_of_purchases = len(purchase_data['Purchase ID'])
number_of_purchases

780

In [232]:
#define total revenue 

total_revenue = purchase_data['Price'].sum()
pd.to_numeric(total_revenue)
TotalRevenueinUSD = ('${:,.2f}'.format(total_revenue))
print(TotalRevenueinUSD)

$2,379.77


In [233]:
#Now with all elements of the Purchasing Analysis, create a DF using dictionary
totals_dicts = [{'Number of Unique Items' : number_of_unique_items_in_df,
                 'Average Price': AveragePriceinUSD,
                 'Number of Purchases' : number_of_purchases,
                 'Total Revenue' : TotalRevenueinUSD,
}]
df_totals = pd.DataFrame(totals_dicts)
df_totals

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$3.05,780,183,"$2,379.77"


In [234]:
#Create data frames including the SN unique count, gender associated, and percentage
#Percentage will be calculated as gender unique count / SN unique count 
# ie 484 unique screen names are Male, 81 are Female, 11 are other
genders_df = pd.DataFrame (purchase_data[['SN', 'Gender']])
gender_percents_value = (purchase_data['Gender'].value_counts() / len(purchase_data ['Gender']))

gendercounts_df = pd.DataFrame(
                    genders_df.groupby("Gender")                             
                    .agg({"SN": "nunique"})
)

gendercounts_df = pd.concat ([gendercounts_df, gender_percents_value], axis=1, join='outer', ignore_index=False, sort=True)
gendercounts_df = gendercounts_df.rename(columns={"SN": "Total Count", "Gender": "Percentage of Players"})
gendercounts_df

,Total Count,Percentage of Players
Female,81,0.144872
Male,484,0.835897
Other / Non-Disclosed,11,0.019231


In [235]:
#Create df's for purchase analysis 

#   total purchase count
#   avg_purchase_price
#   total_purchase_value
#   avg_total_per_person

#1) 
purchase_count = purchase_data_df.groupby("Gender", as_index=False).agg({
    "Price":"count"})
purchase_count

avg_purchase_price = purchase_data_df.groupby("Gender", as_index=False).agg({
    "Price" : "mean"})
avg_purchase_price

avg_purchase_price_df = pd.DataFrame(avg_purchase_price)
avg_purchase_price_df = avg_purchase_price_df.rename(columns={"Price": "Average Purchase Price"})
avg_purchase_price_df["Average Purchase Price"] = avg_purchase_price_df["Average Purchase Price"].map("${:,.2f}".format)
avg_purchase_price_df

purchase_count_df = pd.DataFrame(purchase_count)
purchase_count_df = purchase_count_df.rename(columns={"Price":"Purchase Count"})
purchase_count_df

mergeCount_Price = pd.merge(purchase_count_df, avg_purchase_price_df, on="Gender")
mergeCount_Price

##
#   total_purchase_value = avg_purchase_price_df.groupby("Gender", as_index=False).agg({
#        (purchase_data['Price'])
#   })

,Gender,Purchase Count,Average Purchase Price
0,Female,113,$3.20
1,Male,652,$3.02
2,Other / Non-Disclosed,15,$3.35


In [269]:
#Age Demographics 

# 1) Establish bins for ages 

age_bins= [0,10,15,20,25,30,35,40,max(purchase_data["Age"])+1]
age_output_labels = ['>10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

age_df = pd.cut (x = purchase_data["Age"], bins = age_bins, labels = age_output_labels)
age_df

total_counts = age_df.value_counts()
total_counts

percent_counts = (total_counts)/(len(age_df))
percent_counts_format = percent_counts.map("{:,.2f}".format)

age_summary_table = pd.DataFrame ({
    'Total Counts': total_counts,
    'Percentage of Players': percent_counts_format,
})
age_summary_table.set_index = age_output_labels
age_summary_table

,Total Counts,Percentage of Players
20-24,325,0.42
15-19,200,0.26
25-29,77,0.10
10-14,54,0.07
30-34,52,0.07
35-39,33,0.04
>10,32,0.04
40+,7,0.01


In [285]:
# Age Summary Purchases

#  May use loc for price, avg spend by sn, avg spend on a particular item, total spend per player

age_bins= [0,10,15,20,25,30,35,40,max(purchase_data["Age"])+1]
age_output_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

purchase_data['age_groups'] = pd.cut (x = purchase_data["Age"], bins = age_bins, labels = age_output_labels, right=False)

age_group_count = purchase_data.groupby('age_groups')['age_groups'].count()

age_group_avg_purchase = purchase_data.groupby('age_groups')['Price'].mean()
age_group_avg_purchase_format = age_group_avg_purchase.map("${:,.2f}".format)

age_group_total_purchase = purchase_data.groupby('age_groups')['Price'].sum()
age_group_total_purchase_format = age_group_total_purchase.map("${:,.2f}".format)

#age_group_avg_purchase_per_person = (purchase_data.groupby('age_groups')['SN'].value_counts())
#age_group_avg_purchase_per_person

Age_Groups_DF = pd.DataFrame({
                'Purchase Count': age_group_count, 
                'Average Purchase Price': age_group_avg_purchase_format,
                'Total Purchase Value': age_group_total_purchase_format,
                #'Avg Total Purchase per Person': age_group_avg_purchase_per_person
})

Age_Groups_DF

,Purchase Count,Average Purchase Price,Total Purchase Value
age_groups,,,
<10,23,$3.35,$77.13
10-14,28,$2.96,$82.78
15-19,136,$3.04,$412.89
20-24,365,$3.05,"$1,114.06"
25-29,101,$2.90,$293.00
30-34,73,$2.93,$214.00
35-39,41,$3.60,$147.67
40+,13,$2.94,$38.24


In [298]:
player_purchase_item = purchase_data.groupby ('Item Id' and 'Item Name')['Item Name'].count()

player_item_price = purchase_data.groupby('Item ID' and 'Item Name')['Price'].mean()
player_item_price_format = player_item_price.map("${:,.2f}".format)

player_purchase_value = purchase_data.groupby('Item ID' and 'Item Name')['Price'].sum()
player_purchase_value_format = player_purchase_value.map("${:,.2f}".format)

player_purchase_item = pd.DataFrame({
    
    'Purchase Count': player_purchase_item,
    'Item Price': player_item_price_format,
    'Total Purchase Value': player_purchase_value_format
})

player_purchase_item = player_purchase_item.sort_values ('Purchase Count', ascending=False)
player_purchase_item.head()

,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,13,$4.61,$59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
Persuasion,9,$3.22,$28.99
Nirvana,9,$4.90,$44.10
"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [299]:
player_purchase_item_profit = player_purchase_item.sort_values ('Total Purchase Value', ascending=False)
player_purchase_item.head()

,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,13,$4.61,$59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
Persuasion,9,$3.22,$28.99
Nirvana,9,$4.90,$44.10
"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
